In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import numpy as np
import re
import statsmodels.api as sm
print("success")

success


In [7]:
if os.getcwd()[-9:] == "notebooks":
    os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/home/koos/Documents/timeseries_transfer_learning'

In [ ]:
def normalize_list(lst):
    lst = np.array(lst)
    min_value = min(lst)
    max_value = max(lst)
    if min_value == max_value:
        return lst, (min_value, max_value)
    else:
        return list(((lst - min_value) / (max_value - min_value))), (min_value, max_value)

In [ ]:
def get_df(filepath):
    with open(filepath, "r") as file:
        log = file.read()
    log_lst = []
    for el in log.split("#####################################################"):
        if len(el) > 10:
            log_lst.append(el)
    logs = []
    for el in log_lst:
        logs.append([])
        sub_elms = el.split("\n")

        extracted_content = re.search(r"\{.*\}", sub_elms[9]).group()
        extracted_content = extracted_content.replace("'", "\"")
        params_dict = json.loads(extracted_content)

            
        result_dict = {}

        for item in sub_elms[1:9]:
            parts = item.split(': ')
            key = parts[0].split(' for ')[1].replace('()', '')  # Extracting the key
            value = float(parts[1])  # Extracting the numerical value
            result_dict[key] = value

        logs[-1].append(params_dict) 
        logs[-1].append(result_dict) 
    df_builder = []
    for key, val in logs:
        key.update(val)
        df_builder.append(key)

    df = pd.DataFrame(df_builder)
    return df
df_stock = get_df("/home/koos/Desktop/optuna_logs/STOCK_DATA_STUDY_1701992411.5544963.txt")
df_all = get_df("optuna_logs/ALL_DATA_STUDY_regularization_1704237277.2332232.txt")
df_all_2 = get_df("/home/koos/Desktop/optuna_logs/ALL_DATA_STUDY_1702241361.8004432.txt")

df_all["enrich_ratio"] = df_all["enrich_ratio"] / 10
df_all_2["enrich_ratio"] = df_all_2["enrich_ratio"] / 10

df_stock = df_stock[df_stock["L1Loss"] < 0.1]
df_all = df_all[df_all["L1Loss"] < 0.1]
df_all_2 = df_all_2[df_all_2["L1Loss"] < 0.1]

In [ ]:
losses = ["MSELoss", "HuberLoss"] #"L1Loss", #, "LastValueError", "ArithmetricMeanTotalReturnError", "GeometricMeanDailyReturnError", "MedianFinalReturnError"]#, "OverUnderEstimationError"]
other = "enrich_ratio"

for other in ["enrich_ratio", 'window_size', 'learning_rate', 'num_stacks', 'num_blocks',
       'num_layers', 'expansion_coefficient_dimension',
       'trend_polynomial_degree', 'layer_widths', 'dropout', 'activation']:
    for loss in losses:
        sample = df_all.groupby(other)[loss].median().reset_index()
        plt.plot(sample[other], sample[loss], label=loss)

    plt.title("Loss for models with all data")
    plt.xlabel(other)
    plt.ylabel("loss")
    plt.legend()
    plt.show()


In [ ]:
df_stock.sort_values(by="L1Loss")

In [ ]:
df_all.sort_values(by="L1Loss")

In [ ]:
df_all_2.sort_values(by="L1Loss")

In [ ]:
lr = df_stock.sort_values(by="L1Loss").learning_rate.values
loss = df_stock.sort_values(by="L1Loss").L1Loss.values
plt.scatter(loss, lr)

In [ ]:
df = df_all_2

X = sm.add_constant(df[["enrich_ratio", "window_size", "learning_rate", "num_stacks", "num_blocks", "num_layers", "expansion_coefficient_dimension", "trend_polynomial_degree", "layer_widths"]])
y = df['L1Loss']


# Fitting the linear regression model
model = sm.OLS(y, X).fit()

model.summary()